REQUIRED LIBRARY

LOADING DATASET

In [ ]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt
from keras.models import load_model
from PIL import Image
from sklearn.model_selection import train_test_split
import os
from sklearn.model_selection import cross_val_score, KFold
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def extract_face(filename, required_size=(256, 256)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
# resize pixels to the model size
    image = Image.fromarray(pixels)
    image = image.resize(required_size)
    face_array = np.asarray(image)
    return face_array

In [ ]:
import os

def load_face(dir):
    faces = list()
    # enumerate files
    for filename in os.listdir(dir):
        path = os.path.join(dir, filename)
        face = extract_face(path)
        faces.append(face)
    return faces

def load_dataset(dir, augment=False):
    # list for faces and labels
    X, y = list(), list()
    for subdir in os.listdir(dir):
        path = os.path.join(dir, subdir)
        faces = load_face(path)
        labels = [subdir for i in range(len(faces))]
        print("loaded %d sample for class: %s" % (len(faces), subdir))  # print progress
        
        if augment:
          
            data_gen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
            for face in faces:
                # Expand the dimensions to make it suitable for the flow() function
                face = np.expand_dims(face, axis=0)
                # Generate augmented images and labels
                for x_batch, y_batch in data_gen.flow(face, [subdir], batch_size=1):
                    X.append(x_batch[0])
                    y.append(y_batch[0])
                    break
        else:
            X.extend(faces)
            y.extend(labels)
    
    return np.asarray(X), np.asarray(y)

In [ ]:
# load train dataset
trainX, trainy = load_dataset(r"C:\Users\WELCOME\Desktop\jupiter test\Destination\train")
print(trainX.shape, trainy.shape)

In [ ]:
# load test dataset
testX, testy = load_dataset(r"C:\Users\WELCOME\Desktop\jupiter test\Destination\test")
print(testX.shape, testy.shape)

COMPRESS AND SAVE TRAIN AND TEST DATA

In [ ]:
np.savez_compressed('face.npz', trainX, trainy, testX, testy)

LOAD SAVED COMPRESSED MODEL

In [ ]:
data = np.load("face.npz")
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)

In [ ]:
trainx, valid = train_test_split(trainX, test_size=0.1, random_state=42, shuffle=True)

In [ ]:
print("number of image in train dataset : %s" %(len(trainx)))

print("number of image in Valid  dataset : %s" %(len(valid)))

In [ ]:
y_train, y_valid = train_test_split(trainy, test_size=0.1, random_state=42, shuffle=True)

In [ ]:
print("number of image in train dataset : %s" %(len(y_train)))

print("number of image in Valid dataset : %s" %(len(y_valid)))

In [ ]:
np.savez_compressed('extracface.npz', trainx, y_train, valid, y_valid,testX, testy)

In [ ]:
data = np.load("extracface.npz")
trainx, y_train, valid, y_valid,testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'], data['arr_4'], data['arr_5']
print('Loaded: ', trainx.shape, y_train.shape, valid.shape, y_valid.shape,testX.shape, testy.shape)

FACE EMBEDDING WITH  FACENET

In [ ]:
from keras_facenet import FaceNet
facenet_model=FaceNet();
print('Loaded model')

In [ ]:
print('Loaded: ', trainx.shape, y_train.shape, valid.shape, y_valid.shape,testX.shape, testy.shape)

In [ ]:
def get_embedding(model, face):
    
    face = face.astype('float32')

    #mean, std = face.mean(), face.std()
    #face = (face-mean)/std
    
    sample = np.expand_dims(face, axis=0)
    
    yhat = model.embeddings(sample)
    return yhat[0]

CONVERT EACH FACE IN THE TRAINING SET

In [ ]:
emdTrainX = list()
for face in trainx:
    emd = get_embedding(facenet_model, face)
    emdTrainX.append(emd)
emdTrainX = np.asarray(emdTrainX)
print(emdTrainX.shape)          
embValid = list()
for face in valid:
    emd = get_embedding(facenet_model,face)
    embValid.append(emd)
embValid = np.asarray(embValid)
print(embValid.shape)

CONVERT EACH FACE IN THE TESTING SET

In [ ]:
emdTestX = list()
for face in testX:
    emd = get_embedding(facenet_model, face)
    emdTestX.append(emd)
emdTestX = np.asarray(emdTestX)
print(emdTestX.shape)

COMPRESS AND SAVE TRAIN AND TESTING MODEL

In [ ]:
np.savez_compressed('augembeddings.npz', emdTrainX, y_train, embValid, y_valid, emdTestX, testy)

LOAD SAVED COMPRESSED FILE

In [ ]:
data = np.load('augembeddings.npz')
emdTrainX, y_train, embValid, y_valid, emdTestX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3'], data['arr_4'], data['arr_5']

In [ ]:
print('Loaded: ', emdTrainX.shape, y_train.shape, embValid.shape, y_valid.shape, emdTestX.shape, testy.shape)

LABLE ENCODDING

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle
print("Dataset: train=%d,validation = %d, test=%d" % (emdTrainX.shape[0],embValid.shape[0] ,emdTestX.shape[0]))
# normalize input vectors
in_encoder = Normalizer(norm='l2')
emdTrainX_norm = in_encoder.transform(emdTrainX)
embValid_norm = in_encoder.transform(embValid)
emdTestX_norm = in_encoder.transform(emdTestX)
# label encode targets
out_encoder = LabelEncoder()
encoder_arr = np.append (y_train, 'wangnan')
out_encoder.fit(encoder_arr)

ENCODING

In [ ]:
trainy_enc = out_encoder.transform(y_train)
y_valid_enc = out_encoder.transform(y_valid)
testy_enc = out_encoder.transform(testy)

FACE RECOGNITION WITH SVC

In [ ]:
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
model = SVC(kernel='poly', probability=True)  # You can choose a different kernel if needed

accuracy_scores = cross_val_score(model, emdTrainX, trainy_enc, cv=kf, scoring='accuracy')

# Calculate the average accuracy and standard deviation of the accuracy
average_accuracy = np.mean(accuracy_scores) * 100
std_accuracy = np.std(accuracy_scores) * 100

print("Cross-Validation Accuracies:")
for i, acc in enumerate(accuracy_scores):
    print(f"Fold {i + 1}: {acc * 100:.2f}%")

print(f'\nAverage Cross-Validation Accuracy: {average_accuracy:.2f}%')
print(f'Standard Deviation of Cross-Validation Accuracy: {std_accuracy:.2f}%')

# Train the model on the full training data
model.fit(emdTrainX, trainy_enc)

# Save the trained model
filename = 'Cross_aug.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open('Cross_aug.sav', 'rb'))

In [ ]:

import pickle
from sklearn.metrics import precision_recall_fscore_support

# Evaluate the model on the test set
predictions = loaded_model.predict(emdTestX)

# Calculate precision, recall, and F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(testy_enc, predictions, average='weighted')

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1_score:.2f}")


In [ ]:
from random import choice
for i in range(20):
    # select a random face from test set
    selection = choice([i for i in range(testX.shape[0])]) 
    random_face = testX[selection]
    random_face_emd = emdTestX_norm[selection]
    random_face_class = testy_enc[selection]
    random_face_name = out_encoder.inverse_transform([random_face_class])
    # prediction for the face
    samples = np.expand_dims(random_face_emd, axis=0)
    yhat_class = loaded_model.predict(samples)
    yhat_prob = loaded_model.predict_proba(samples)
    class_index = yhat_class[0]
    if class_index <= 75:
        # get name
        class_probability = yhat_prob[0,class_index] * 100
        predict_names = out_encoder.inverse_transform(yhat_class)
        #print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
        #if random_face_name[0] == predict_names[0]:
        print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
        print('Expected: %s' % random_face_name[0])
        # plot face
        plt.imshow(random_face)
        title = '%s (%.3f)' % (predict_names[0], class_probability)
        plt.title(title)
        plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_accuracy_graph(accuracies):
    num_folds = len(accuracies)
    fold_indices = range(1, num_folds + 1)

    plt.bar(fold_indices, accuracies, color='skyblue')
    plt.xlabel('Fold')
    plt.ylabel('Accuracy')
    plt.title('Accuracy for each fold in 5-fold cross-validation')
    plt.xticks(fold_indices)
    plt.ylim(0, 1.0)  # Adjust the y-axis limit based on your accuracy range (usually [0, 1])
    plt.grid()

    plt.show()

def plot_mean_accuracy_graph(mean_accuracy):
    plt.bar(['Mean Accuracy'], [mean_accuracy], color='skyblue')
    plt.xlabel('Metric')
    plt.ylabel('Accuracy')
    plt.title('Mean Accuracy in 5-fold cross-validation')
    plt.ylim(0, 1.0)  # Adjust the y-axis limit based on your accuracy range (usually [0, 1])
    plt.grid()

    plt.show()

# Sample data for accuracy values for each fold (replace this with your actual data)
sample_accuracies = [0.98, 0.97, 0.98, 0.96, 0.97]

# Sample mean accuracy value (replace this with your actual mean accuracy value)
sample_mean_accuracy = np.mean(sample_accuracies)

# Plot accuracy for each fold
plot_accuracy_graph(sample_accuracies)



In [ ]:
yhat_valid = loaded_model.predict(embValid_norm)
yhat_test = loaded_model.predict(emdTestX_norm)

In [ ]:
recall_valid = recall_score(y_valid_enc, yhat_valid, average='weighted')
precision_valid = precision_score(y_valid_enc, yhat_valid, average='weighted')
f1_valid = f1_score(y_valid_enc, yhat_valid, average='weighted')

recall_test = recall_score(testy_enc, yhat_test, average='weighted')
precision_test = precision_score(testy_enc, yhat_test, average='weighted')
f1_test = f1_score(testy_enc, yhat_test, average='weighted')

# Print confusion matrix, accuracy, recall, precision, and F1-score
conf_matrix_valid = confusion_matrix(y_valid_enc, yhat_valid)
conf_matrix_test = confusion_matrix(testy_enc, yhat_test)

print('Validation Set:')
print('Accuracy: %.3f' % accuracy_valid)
print('Recall: %.3f' % recall_valid)
print('Precision: %.3f' % precision_valid)
print('F1-score: %.3f' % f1_valid)
print('Confusion Matrix:')
print(conf_matrix_valid)

print('------------------------')

print('Test Set:')
print('Accuracy: %.3f' % accuracy_test)
print('Recall: %.3f' % recall_test)
print('Precision: %.3f' % precision_test)
print('F1-score: %.3f' % f1_test)
print('Confusion Matrix:')
print(conf_matrix_test)